# Solution for problem 5 of reference [11]

General setup.

In [1]:
# Imports
import scipy.integrate as integrate
import numpy as np

Define the parameters like they did in [this paper](https://arxiv.org/abs/2303.12107).

In [2]:
# Constants
r_earth = 8.2 # kpc
rho_earth = 0.42 # GeV/cm^3
# I think these parameters could be improved, they don't seem to be the best fit for the Milky Way
r_s = 18.6 # approximate value based on what I remember from the call, kpc
rho_s = rho_earth / (r_s / r_earth) / (1 + r_earth / r_s) ** (-2) # definition to make rho_earth 0.3, GeV/cm^3
r_0 = 0.5 # kpc

Define NFW distributions, $r$ as a function of $s$ and $\theta$, and $\Delta \Omega$.

In [3]:
def rho_NFW(r):
	# Returns in GeV/cm^3
	return rho_s * (r_s / r) * (1 + r / r_s) ** (-2)

def rho_NFW_core(r):
	# Returns in GeV/cm^3
	return rho_s * (r_s / (r + r_0)) * (1 + r / r_s) ** (-2)

def r(s, theta):
	# Returns in kpc
	return np.sqrt(r_earth**2 + s**2 - 2*r_earth*s*np.cos(theta))

def deltaOmega(theta_1, theta_2):
	# Returns in steradians
	return 2 * np.pi * (np.cos(theta_1) - np.cos(theta_2))

Define integrand for the $J$ integral, the function $J(\theta)$, the integrand for the $\bar{J}$ integral and the function $\bar{J}(\theta_1, \theta_2)$. They are all dimensionless.

In [4]:
def J_integrand(s, theta, rho_DM):
	return (rho_DM(r(s, theta)) / rho_earth) ** 2 / r_earth

def J(theta, rho_DM):
	integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta, rho_DM))
	return integral

def J_avg_integrand(theta, rho_DM):
	return J(theta, rho_DM) * np.sin(theta)

def J_avg(theta_1, theta_2, rho_DM):
	integral, _ = integrate.quad(J_avg_integrand, theta_1, theta_2, args=(rho_DM,))
	return integral * 2 * np.pi / deltaOmega(theta_1, theta_2)

## Questions 1 and 2

Use previously defined functions to calculate $\bar{J}$ for different distributions.

In [5]:
# Integrating from 0 gives convergence issues, it's probably because
# of the divergence of the NFW profile. This can easily be fixed by
# starting integration from an angle close to zero.
J_avg_NFW = J_avg(0, np.pi / 180, rho_NFW)
J_avg_NFW_core = J_avg(0, np.pi / 180, rho_NFW_core)

print("J_avg NFW: ", J_avg_NFW)
print("J_avg NFW core: ", J_avg_NFW_core)

C:\Users\lucaz\AppData\Local\Temp\ipykernel_21508\2604824896.py:5: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta, rho_DM))
C:\Users\lucaz\AppData\Local\Temp\ipykernel_21508\2604824896.py:5: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  integral, _ = integrate.quad(J_integrand, 0, np.inf, args=(theta,

J_avg NFW:  1477.7654169594402
J_avg NFW core:  104.64977923429672


C:\Users\lucaz\AppData\Local\Temp\ipykernel_21508\2604824896.py:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral, _ = integrate.quad(J_avg_integrand, theta_1, theta_2, args=(rho_DM,))


From Eq. (6.4) at page 196 of the [review by Cirelli, Strumia and Zupan](https://arxiv.org/abs/2406.01705v2), I understand that $\langle \sigma v \rangle \propto \frac{1}{J}$. This means that a higher J-factor leads to a more stringent (smaller) upper bound on the DM annihilation cross section. Based on the numerical estimates shown above, there's a difference of roughly two orders of magnitude between the value obtained with the standard NFW distribution and the one derived from the cored NFW distribution. This translates into an upper bound for the standard NFW distribution that is one/two orders of magnitude lower than the one for the cored NFW distribution.  
By quickly skimming through this paper, it seems like my observation makes sense: [Constraints on an Annihilation Signal from a Core of Constant Dark Matter Density around the Milky Way Center with H.E.S.S.](https://arxiv.org/abs/1502.03244).

## Question 3

This question refers to the following paper: [Search for dark matter annihilations towards the inner Galactic halo
from 10 years of observations with H.E.S.S.](https://arxiv.org/abs/1607.08142). I first calculate the new J-factors for both distributions averaged over the annulus $[0.3°, 1.0°]$.

In [6]:
J_avg_NFW = J_avg(0.3 * np.pi / 180, np.pi / 180, rho_NFW)
J_avg_NFW_core = J_avg(0.3 * np.pi / 180, np.pi / 180, rho_NFW_core)

print("J_avg NFW: ", J_avg_NFW)
print("J_avg NFW core: ", J_avg_NFW_core)

J_avg NFW:  1119.3534400097778
J_avg NFW core:  104.00592917743928


Therefore I expect the potential error to be of roughly one order of magnitude (with the standard NFW upper bound lower than the cored NFW one).

## Question 4

Done on paper.